In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import sys
print(sys.version)
sys.path.append("../hands_on/interprettensor/interprettensor")
sys.path.append("../hands_on/interprettensor/interprettensor/modules")
from modules.sequential import Sequential
from modules.linear import Linear
from modules.softmax import Softmax
from modules.relu import Relu
from modules.tanh import Tanh
from modules.convolution import Convolution
import modules.render as render
from tensorflow.examples.tutorials.mnist import input_data #import input_data
from modules.utils import Utils, Summaries, plot_relevances

import argparse
import tensorflow as tf
import numpy as np
import pdb as pdb
import os

flags = tf.flags
logging = tf.logging

flags.DEFINE_integer("batch_size", 4,'Number of steps to run trainer.')
flags.DEFINE_string("data_dir", 'data','Directory for storing data')
flags.DEFINE_string("summaries_dir", 'my_model_logs','Summaries directory')
flags.DEFINE_boolean("relevance", True,'Compute relevances')
flags.DEFINE_string("checkpoint_dir", 'mnist_linear_model','Checkpoint dir')


FLAGS = flags.FLAGS
print('Imported')

3.4.3 (default, Nov 17 2016, 01:08:31) 
[GCC 4.8.4]
Imported


In [2]:
def neural_network():
    # Define the operation to be performed
    with tf.variable_scope('XOR'):
                    
        return Sequential([Linear(input_dim=2,output_dim=200, act='relu', batch_size=FLAGS.batch_size),
                           Linear(1,act='sigmoid')
                          ])
        

In [3]:
''' Tensorflow codes - XOR'''

# XOR 
# 00 - 0
# 01 - 1
# 10 - 1
# 11 - 0

tf.reset_default_graph()
# Define the inputs 
with tf.variable_scope('inputs/ab'):
    ab = tf.placeholder(tf.float32, shape=[4,2])
with tf.variable_scope('ground_truth'):
    ground_truth= tf.placeholder(tf.float32, shape=[4,1])


with tf.variable_scope('model'):
    nn = neural_network()
    output = nn.forward(ab)
    relevances = nn.lrp(output)
#print(ab,hidden_layer,output)

# Define loss function and optimizers
learning_rate = 0.10
with tf.variable_scope('mean_squared_error'):
     loss = tf.losses.mean_squared_error(ground_truth, output)
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)


Forward Pass ... 
------------------------------------------------- 
linear_1:: [4, 2]
linear_2:: [4, 200]

------------------------------------------------- 
Computing LRP ... 
------------------------------------------------- 
linear_2:: [4, 200]
linear_1:: [4, 2]

------------------------------------------------- 


In [4]:
#Create a tensorflow session
with tf.Session() as sess:
    # initialize all the variables - this assigns the values to the variables
    tf.global_variables_initializer().run()
    
    # Create the XOR table
    input_ = np.array([[0,0],[0,1],[1,0],[1,1]]).reshape([4,2])
    gt_ = np.array([0,1,1,0]).reshape([4,1])
    
    # The dictionary assigns the numerical values to the placeholders
    values_dict = {ab:input_, ground_truth:gt_}
    
    max_steps = 1000
    for i in range(max_steps):
        # sess.run() consists of 'fetch' and 'feed' 
        # 'fetch' are the list of variables to be returned
        # 'feed' is the dictionary of values assigned to respective placeholders
        _, input1, output_, loss_,relevances_ = sess.run([optimizer, ab,output, loss, relevances], feed_dict=values_dict)
    print(i,loss_)
    

999 1.6943e-05


In [5]:
for i in range(4):
    #print("Input - a: %d:, b: %d - output: %g"%(input_[i,0], input_[i,1], output_[i]))
    
    #
    print("Input - a: %d:, b: %d - output: %g - relevances_a: %g, relevances_b: %g"%(input_[i,0], input_[i,1], 
                                                            output_[i], relevances_[i,0], relevances_[i,1]))
            
    

Input - a: 0:, b: 0 - output: 0.000572218 - relevances_a: 0, relevances_b: 0
Input - a: 0:, b: 1 - output: 0.998759 - relevances_a: 0, relevances_b: 0.839912
Input - a: 1:, b: 0 - output: 0.999396 - relevances_a: 1.90762, relevances_b: 0
Input - a: 1:, b: 1 - output: 0.00809571 - relevances_a: 0, relevances_b: 0
